In [45]:
import numpy as np
from scipy import integrate

# All constants and comments describing them are from Widiasih, referenced in references.bib
# TODO: Create proper bibliography with BibTeX in Markdown cell

Q = 343 # Current value of solar insolation
ocean_albedo = 0.32 # Amount of radiation reflected by this surface
ice_albedo = 0.62 # Amount of radiation reflected by this surface
C = 3.04 # Constant traditionally chosen so that one of the equilibria fits the current climate with an ice line near the pole
critical_temperature = -10 # Celsius value at which liquid water will freeze
R = 12.6 # Heat capacity of the surface of the planet
M = 25 # Steepness of the albedo function near the ice line
epsilon = 0.001 # Constant to control rate of ice line movement

# approximation of both the Stephan-Boltzman's law of black body radiation and the greenhouse gas effect on the atmosphere (A + BT)
A = 202
B = 1.9

num_of_latitude_steps = 200
latitude_range = np.linspace(0, 1, num_of_latitude_steps) # 0 to 1 goes from the equator to the pole in y
initial_temperature_equator = 35 # Sample Celsius from weather on 8/2/2023
initial_temperature_pole = -2 # Sample Celsius
initial_temperature_profile = np.linspace(initial_temperature_equator, initial_temperature_pole, num_of_latitude_steps)

num_of_time_steps = 1000
time_beginning = 0
time_end = 10
delta_time = (time_end - time_beginning) / num_of_time_steps
time_range = np.linspace(time_beginning, time_end, num_of_latitude_steps)

initial_eta = 0.2 # eta at 0 is snowball, eta at 1 is no ice

temperatures_matrix = np.empty((num_of_latitude_steps, num_of_time_steps))
temperatures_matrix[:, 0] = initial_temperature_profile
eta_matrix = np.empty(num_of_time_steps)
eta_matrix[0] = initial_eta

def diff_equ_budyko(eta, y, T, Q, ocean_albedo, ice_albedo, C, R, M, current_time_step, temperatures_matrix):
    """Returns the result of the continuous differential equation referred to related to equation 2.5"""
    s = legendre_latitude_distr(y)
    albedo = ice_line_dependent_albedo(eta, y, ocean_albedo, ice_albedo, M)
    integral = integral_temp_profile(temperatures_matrix[:, current_time_step - 1], y)

    return_value = (1 / R) * (Q * s * (1 - albedo) - (A + B * T) - C * (T - integral))
    return return_value

def legendre_latitude_distr(y):
    """The function referred to as s(y) calculating the distribution of radiation based on latitude."""
    return 1 - (0.482 * (((3 * (y ** 2)) - 1) / 2))

def ice_line_dependent_albedo(eta, y, ocean_albedo, ice_albedo, M):
    """Returns the albedo at location y."""
    return (ocean_albedo + ice_albedo) / 2 + ((ice_albedo - ocean_albedo) / 2) * np.tanh(M * (y - eta))

def integral_temp_profile(T_i, y):
    """Returns the integral from 0 to 1 of the previous column of """
    return integrate.trapezoid(T_i, y)

def ice_line_equation(eta, critical_temperature, epsilon, current_time_step, temperatures_matrix):
    """Returns the result of the ice line equation at eta, representing the change in the ice line."""
    T_i = temperatures_matrix[:, current_time_step - 1]
    eta_temp = T_i[round(eta * num_of_latitude_steps) - 1] # May have to check again if this is correct
    return epsilon * (eta_temp - critical_temperature)

for i in range(1, num_of_time_steps):
    new_eta = eta_matrix[i - 1] + ice_line_equation(eta_matrix[i - 1], critical_temperature, epsilon, i, temperatures_matrix)
    print(new_eta)
    # New etas are bounded between 0 and 1
    if new_eta < 0:
        eta_matrix[i] = 0
    elif new_eta > 1:
        eta_matrix[i] = 1
    else:
        eta_matrix[i] = new_eta
    temperatures_matrix[:, i] = temperatures_matrix[:, i - 1] + diff_equ_budyko(eta_matrix[i], latitude_range, temperatures_matrix[:, i - 1], Q, ocean_albedo, ice_albedo, C, R, M, i, temperatures_matrix)

eta_matrix

0.23774874371859298
0.3091617953113545
0.4130328871245128
0.5522867676574912
0.7317489627595279
0.9537059389909099
1.2193773588471055
1.3349940755653336
1.4230940123879274
1.5257164751394612
1.6449393938492456
1.783164168243761
1.9431635283051334
2.128136694830064
2.341772933279552
2.5883247597498578
2.872692248336276
3.2005201082637917
3.5783094512497176
4.013546459621908
4.514850499471612
5.092144607165437
5.756851719463207
6.522120526018189
7.403085408220176
8.417165601741843
9.58440949506925
10.927890868049385
12.474164900690447
14.253792963380365
16.301946558679283
18.659102348744238
21.37184200215986
24.49377266503806
28.086586244634518
32.221278436504086
36.97955158257504
42.455429079830395
48.757113239314045
56.00912330552432
64.35475588196498
73.95891637907553
85.0113774318327
97.7305286708589
112.36769193959066
129.21208722278607
148.59654740914866
170.90409480717008
196.57550936080662
226.11803810675698
260.1154179650815
299.2394099052677
344.26307239335864
396.0760363925013

array([0.2       , 0.23774874, 0.3091618 , 0.41303289, 0.55228677,
       0.73174896, 0.95370594, 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.     

```{bibliography}
```